In [57]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 20: Infinite Elves and Infinite Houses ---</h2><p>To keep the Elves busy, Santa has them deliver some presents <span title="This was before the Elves unionized, apparently.">by hand, door-to-door</span>.  He sends them down a street with infinite houses numbered sequentially: <code>1</code>, <code>2</code>, <code>3</code>, <code>4</code>, <code>5</code>, and so on.</p>
<p>Each Elf is assigned a number, too, and delivers presents to houses based on that number:</p>
<ul>
<li>The first Elf (number <code>1</code>) delivers presents to every house: <code>1</code>, <code>2</code>, <code>3</code>, <code>4</code>, <code>5</code>, ....</li>
<li>The second Elf (number <code>2</code>) delivers presents to every second house: <code>2</code>, <code>4</code>, <code>6</code>, <code>8</code>, <code>10</code>, ....</li>
<li>Elf number <code>3</code> delivers presents to every third house: <code>3</code>, <code>6</code>, <code>9</code>, <code>12</code>, <code>15</code>, ....</li>
</ul>
<p>There are infinitely many Elves, numbered starting with <code>1</code>.  Each Elf delivers presents equal to <em>ten times</em> his or her number at each house.</p>
<p>So, the first nine houses on the street end up like this:</p>
<pre><code>House 1 got 10 presents.
House 2 got 30 presents.
House 3 got 40 presents.
House 4 got 70 presents.
House 5 got 60 presents.
House 6 got 120 presents.
House 7 got 80 presents.
House 8 got 150 presents.
House 9 got 130 presents.
</code></pre>
<p>The first house gets <code>10</code> presents: it is visited only by Elf <code>1</code>, which delivers <code>1 * 10 = 10</code> presents.  The fourth house gets <code>70</code> presents, because it is visited by Elves <code>1</code>, <code>2</code>, and <code>4</code>, for a total of <code>10 + 20 + 40 = 70</code> presents.</p>
<p>What is the <em>lowest house number</em> of the house to get at least as many presents as the number in your puzzle input?</p>
</article>


In [58]:
# https://oeis.org/A326122
# https://oeis.org/A051444
from math import prod
from typing import Generator
from sympy import divisors

expected_sigma_inverse = [10, 30, 40, 70, 60, 120, 80, 150, 130]


# lowest_housnumber(33_100_000)
def strict_partitions(n: int, index=0) -> Generator[tuple[int, ...]]:
    yield n,
    for i in range(index + 1, n // 2 + 1):
        for p in strict_partitions(n - i, i):
            p1 = (i,) + p
            yield p1


def sigma(n):
    return sum(divisors(n))


def sigma_inverse(n):
    return next((k for k in range(1, n + 1) if sigma(k) == n), 0)


# fmt: off
expected_sigma = [
    1, 3, 4, 7, 6, 12, 8, 15, 13, 18, 12, 28, 14, 24, 24, 31, 
    18, 39, 20, 42, 32, 36, 24, 60, 31, 42, 40, 56, 30, 72, 32, 
    63, 48, 54, 48, 91, 38, 60, 56, 90, 42, 96, 44, 84, 78, 72, 
    48, 124, 57, 93, 72, 98, 54, 120, 72, 120, 80, 90, 60, 168, 
    62, 96, 104, 127, 84, 144, 68, 126, 96, 144
]
# fmt: on

assert all(sigma(i) == expected_sigma[i - 1] for i in range(1, len(expected_sigma) + 1))

# fmt: off
expected_sigma_inverse = [
    1, 0, 2, 3, 0, 5, 4, 7, 0, 0, 0, 6, 9, 13, 8, 0, 0, 10, 0, 19, 
    0, 0, 0, 14, 0, 0, 0, 12, 0, 29, 16, 21, 0, 0, 0, 22, 0, 37, 18, 
    27, 0, 20, 0, 43, 0, 0, 0, 33, 0, 0, 0, 0, 0, 34, 0, 28, 49, 0, 
    0, 24, 0, 61, 32, 0, 0, 0, 0, 67, 0, 0, 0, 30, 0, 73, 0, 0, 0, 
    45, 0, 57, 0, 0, 0, 44, 0, 0, 0, 0, 0 
]
# fmt: on

assert all(
    sigma_inverse(i) == expected_sigma_inverse[i - 1]
    for i in range(1, len(expected_sigma_inverse) + 1)
)


# sigma_inverse(33_100_000 // 10)
# 2_476_767 ==> too high Should be first number with sigma >= n // 10

sigma(2_476_767) * 10 == 33_100_000

True

In [59]:
def first_bigger_then(n):
    n //= 10
    return next((k for k in range(1, n + 1) if sigma(k) >= n), 0)


print("Part I solution =", first_bigger_then(33_100_000))

Part I solution = 776160


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>776160</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The Elves decide they don't want to visit an infinite number of houses.  Instead, each Elf will stop after delivering presents to <code>50</code> houses.  To make up for it, they decide to deliver presents equal to <em>eleven times</em> their number at each house.</p>
<p>With these changes, what is the new <em>lowest house number</em> of the house to get at least as many presents as the number in your puzzle input?</p>
</article>

<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+Part+One+of+%22Infinite+Elves+and+Infinite+Houses%22+%2D+Day+20+%2D+Advent+of+Code+2015&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F20&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+Part+One+of+%22Infinite+Elves+and+Infinite+Houses%22+%2D+Day+20+%2D+Advent+of+Code+2015+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F20';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>


In [60]:
from sympy import divisors


def presents(n: int) -> int:
    return sum(d for d in divisors(n) if n // d <= 50)


def first_bigger_then_II(n):
    n //= 10
    return next((k for k in range(1, n + 1) if 11 * presents(k) >= 10 * n), 0)


print("Part II solution =", first_bigger_then_II(33_100_000))
# 887040 too high

Part II solution = 786240


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>786240</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2015">return to your Advent calendar</a> and try another puzzle.</p>
<p>Your puzzle input was <code class="puzzle-input">33100000</code>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Infinite+Elves+and+Infinite+Houses%22+%2D+Day+20+%2D+Advent+of+Code+2015&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F20&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Infinite+Elves+and+Infinite+Houses%22+%2D+Day+20+%2D+Advent+of+Code+2015+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F20';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
